In [1]:
import pandas as pd
import numpy as np

# For stackoverflow uncomment following lines
csz = '18'
rranks_file = '/data/khodadaa/analysis/stack.csv'
features_file = "/data/khodadaa/ml-exp/stack_feat/{}.csv".format(csz)
features_complement_file = "/data/khodadaa/ml-exp/stack_feat/c{}.csv".format(csz)
ml_file = "/data/khodadaa/ml-exp/stack_feat/ml_in_{}.csv".format(csz)


# For wikipedia uncomment following lines
# def generate_rr_freq_file(filename):
#     msn6 = pd.read_csv('/data/khodadaa/ml-exp/wiki/msn_6.csv', usecols=['query', 'freq', 'label'])
#     msn6.rename(columns={'query': 'Query', 'freq': 'TestViewCount', 'label': 'Y'}, inplace=True)
#     msn6['Y'] = np.where(msn6['Y'] == 0, 'sub', 'all')

#     msn_rr = pd.read_csv('/data/khodadaa/ml-exp/wiki/msn_ml_6.csv', usecols=['query', 'p2%', 'p100%'])
#     msn_rr.rename(columns={'query': 'Query', 'p2%': csz, 'p100%': '100'}, inplace=True)

#     mg_rr = pd.merge(msn6, msn_rr, on='Query', how='inner')
#     mg_rr.to_csv(filename, index=False, columns=['Query', 'TestViewCount', csz, '100'])
# csz = '2'
# rranks_file = '/data/khodadaa/ml-exp/wiki/msn_rranks_frequency.csv'
# features_file = "/data/khodadaa/ml-exp/wiki/msn_{}.csv".format(csz)
# features_complement_file = "/data/khodadaa/ml-exp/wiki/msn_c{}.csv".format(csz)
# ml_file = "/data/khodadaa/ml-exp/wiki/ml_in_{}.csv".format(csz)
# generate_rr_freq_file(rranks_file)


# For inex uncomment following lines
# csz = '24'
# rranks_file = '/data/khodadaa/ml-exp/inex/inex_scores_{}.csv'.format(csz)
# features_file = "/data/khodadaa/ml-exp/inex/inex_{}.csv".format(csz)
# features_complement_file = "/data/khodadaa/ml-exp/inex/inex_c{}.csv".format(csz)
# ml_file = "/data/khodadaa/ml-exp/inex/inex_ml_in_{}.csv".format(csz)

In [2]:
%%time
feat_df = pd.read_csv(features_file)
feat_complement_df = pd.read_csv(features_complement_file)

# drop last column which is blank
feat_df.drop(columns=feat_df.columns[-1:], inplace=True)
feat_complement_df.drop(columns=feat_complement_df.columns[-1:], inplace=True)

rr_df = pd.read_csv(rranks_file).filter(items=['Query', 'TestViewCount', csz, '100'])
# rr_df.rename(columns={csz: 'sub_rr', '100': 'all_rr', 'ql_t': 'sub_ql', 'ql_t.1': 'all_ql'}, inplace=True)

conc_df = pd.merge(feat_df, feat_complement_df, suffixes=('_sub', '_cmp'), validate='1:1', left_index=True, right_index=True)
conc_df = pd.merge(rr_df, conc_df, suffixes=('_rrdf', ''), validate='1:1', left_index=True, right_index=True)

# def get_cache_label_indices(df):
#     cache_size = int(df.shape[0]*(int(csz)/100))
#     break_index = df.sort_values(by='TestViewCount', ascending=False).index[cache_size]
#     return df['TestViewCount'] >= df.loc[break_index, 'TestViewCount']

# c_lbl = get_cache_label_indices(conc_df)
# conc_df['Y'] = np.where(c_lbl, 'sub', 'all')
conc_df['Y'] = np.where(conc_df[csz] >= conc_df['100'], 'sub', 'all')

conc_df.drop(columns=['query_sub', 'query_cmp'], inplace=True)
conc_df.rename(columns={csz: 'rr_sb', '100': 'rr_al'}, inplace=True)
conc_df.to_csv(ml_file, index=False)
print(list(conc_df.columns))

['Query', 'TestViewCount', 'rr_sb', 'rr_al', 'covered_t_sub', 'mean_df_t_sub', 'min_df_t_sub', 'mean_mean_pop_t_sub', 'mean_min_pop_t_sub', 'min_mean_pop_t_sub', 'min_min_pop_t_sub', 'ql_t_sub', 'qll_t_sub', 'covered_t_bi_sub', 'mean_df_t_bi_sub', 'min_df_t_bi_sub', 'mean_mean_pop_t_bi_sub', 'mean_min_pop_t_bi_sub', 'min_mean_pop_t_bi_sub', 'min_min_pop_t_bi_sub', 'ql_t_bi_sub', 'qll_t_bi_sub', 'scs_t_sub', 'maxSCQ_t_sub', 'covered_t_cmp', 'mean_df_t_cmp', 'min_df_t_cmp', 'mean_mean_pop_t_cmp', 'mean_min_pop_t_cmp', 'min_mean_pop_t_cmp', 'min_min_pop_t_cmp', 'ql_t_cmp', 'qll_t_cmp', 'covered_t_bi_cmp', 'mean_df_t_bi_cmp', 'min_df_t_bi_cmp', 'mean_mean_pop_t_bi_cmp', 'mean_min_pop_t_bi_cmp', 'min_mean_pop_t_bi_cmp', 'min_min_pop_t_bi_cmp', 'ql_t_bi_cmp', 'qll_t_bi_cmp', 'scs_t_cmp', 'maxSCQ_t_cmp', 'Y']
CPU times: user 2min 8s, sys: 12.4 s, total: 2min 20s
Wall time: 1min 33s


In [3]:
conc_df.shape

(1092107, 45)